In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
from PIL import Image
from skimage.transform import resize
from sklearn.model_selection import train_test_split

import helper

import datetime as dt
import tensorflow as tf

from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, InceptionV3, decode_predictions
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D


%matplotlib inline

In [20]:
start_time = dt.datetime.now()
NUM_CLASSES = 2
DEBUG = True

### Keras Pretrained Model

There are only average 85 images per label (dog breed). This is a very small dataset to train a CNN from scratch. Moreover pretrained model like inception are supposed to work great for this type of problem.

## Split into Train and Validation set
split the data into 80% train data and 20% validation data.

In [51]:
labels_filename = "./input/labels.csv"

# @todo choose a smaller set of labels for testing the model implementation
labels = pd.read_csv(labels_filename, delimiter=",")
if(DEBUG==True):
    selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
    labels = labels[labels['breed'].isin(selected_breed_list)]
#     print(labels)
train_labels, val_labels = train_test_split(labels, test_size=0.2, random_state=42)
print(train_labels.shape)
print(val_labels.shape)

(194, 2)
(49, 2)


In [37]:
def load_image(image_id, size):
    filename = "./input/train/" + image_id + ".jpg"
    image = resize(np.array(plt.imread(filename)), size)
    return image 
# image = load_image('ffa0055ec324829882186bae29491645', (224,224))
# print(image)

## Inception implementaion from Keras
Based on the implementation example provided by keras site - https://keras.io/applications/ create a generate_arrays method first -

In [49]:
#yield input, output tuples 1 at a time for entire label set
def generate_train_arrays(labels):
    print("inside generate train arrays")
    for label in labels.values:
#         print("image label is ", label)
        image = load_image(label[0], (224, 224))
#         print(image)
        yield(image, label[1])
        
def generate_test_arrays(labels):
    for label in labels.values:
        image = load_image(label[0], (224, 224))
        yield(image)

In [50]:
# This code is taken from keras example of inceptionv3 implementation

base_model = InceptionV3(weights='imagenet', include_top=False)

x=base_model.output

#add a global spatial average pooling layer
x=GlobalAveragePooling2D()(x)

# add a fully connected layer
x = Dense(1024, activation="relu")(x)

# add a logistic layer
x = Dense(NUM_CLASSES, activation="softmax")(x)

# our model
model = Model(inputs=base_model.inputs, outputs=x)

# freeze base_model layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile our model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train our model
model.fit_generator(generate_train_arrays(train_labels), steps_per_epoch=1000, epochs=10)


# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(generate_train_arrays(train_labels), steps_per_epoch=1000, epochs=10)

predictions = model.predict_generator(generate_test_arrays(val_labels))
print("predictions : ", decode_predictions(predictions, top=3)[0])

inside generate train arrays
Epoch 1/10


/anaconda2/envs/ML3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda2/envs/ML3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


ValueError: Error when checking input: expected input_11 to have 4 dimensions, but got array with shape (224, 224, 3)